In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
sz = 128

In [4]:
%%time
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('train',
                                                 target_size=(sz, sz),
                                                 batch_size=10,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('test',
                                            target_size=(sz , sz),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical') 

Found 12845 images belonging to 27 classes.
Found 4262 images belonging to 27 classes.
Wall time: 2.71 s


## Model

In [8]:
# build network topology
model = keras.Sequential([

    # 1st convolution and maxpooling layer
    keras.layers.Conv2D(36, (3, 3), input_shape=(128,128,1), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    # 2nd convolution and maxpooling layer
    keras.layers.Conv2D(36, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    # 3rd convolution and maxpooling layer
    keras.layers.Conv2D(36, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    # Flattening
    keras.layers.Flatten(),

    # full connection layers
#     keras.layers.Dense(1024, activation='relu'),
#     keras.layers.Dropout(0.2),
    keras.layers.Dense(156, activation='relu'),
    keras.layers.Dropout(0.35),
    keras.layers.Dense(96, activation='relu'),
    keras.layers.Dropout(0.35),
    keras.layers.Dense(64, activation='relu'),
    
    # Output layer
    keras.layers.Dense(27, activation='softmax')
])

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimiser,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 126, 126, 36)      360       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 63, 63, 36)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 61, 61, 36)        11700     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 30, 36)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 36)        11700     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 36)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7056)             

In [9]:
training_set.target_size

(128, 128)

In [10]:
%%time

# model.fit_generator(
#         training_set,
#         steps_per_epoch=12841, # No of images in training set
#         epochs=5,
#         validation_data=test_set,
#         validation_steps=4268)# No of images in test set

model.fit(training_set, validation_data=test_set, epochs=5)

# train model
# model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=10, epochs=2)

Epoch 1/5
1285/1285 [==============================] - 330s 257ms/step - loss: 1.4763 - accuracy: 0.5428 - val_loss: 0.0811 - val_accuracy: 0.9833
Epoch 2/5
1285/1285 [==============================] - 174s 135ms/step - loss: 0.3499 - accuracy: 0.8863 - val_loss: 0.0352 - val_accuracy: 0.9920
Epoch 3/5
1285/1285 [==============================] - 121s 94ms/step - loss: 0.2029 - accuracy: 0.9309 - val_loss: 0.0335 - val_accuracy: 0.9901
Epoch 4/5
1285/1285 [==============================] - 160s 124ms/step - loss: 0.1504 - accuracy: 0.9527 - val_loss: 0.0076 - val_accuracy: 0.9977
Epoch 5/5
1285/1285 [==============================] - 136s 106ms/step - loss: 0.1134 - accuracy: 0.9636 - val_loss: 0.0087 - val_accuracy: 0.9974
Wall time: 15min 22s


## Save Model

In [14]:
# Saving the model
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
model.save_weights('model-bw.h5')
print('Weights saved')

Model Saved
Weights saved
